In [23]:
import mir_eval
import numpy as np
import os
import pandas as pd
import sys

sys.path.append("../src")
import localmodule

In [35]:
data_dir = "/Users/vl238/spl2017_data"
dataset_name = localmodule.get_dataset_name()
annotations_name = "_".join([dataset_name, "annotations"])
annotations_dir = os.path.join(data_dir, annotations_name)
skm_predictions_name = "_".join([dataset_name, "baseline-predictions"])
skm_predictions_dir = os.path.join(data_dir, skm_predictions_name)
units = localmodule.get_units()
folds = localmodule.split_units()
n_folds = len(folds)

fold_id = 0
# for fold_id in range(n_folds):


unit_minima = []
unit_maxima = []
for unit_str in units:
    prediction_path = os.path.join(skm_predictions_dir, unit_str + ".npy")
    prediction_matrix = np.load(prediction_path)
    odf = prediction_matrix[:, 1]
    unit_min = np.min(odf)
    unit_minima.append(unit_min)
    unit_max = np.max(odf)
    unit_maxima.append(unit_max)

timestamps = prediction_matrix[:, 0]
annotation_path = os.path.join(annotations_dir, unit_str + ".txt")
df = pd.read_csv(annotation_path, "\t")

TypeError: object of type 'NoneType' has no len()

In [36]:
folds